<a href="https://colab.research.google.com/github/raistra/F-Case-Study-04-Data-Mining/blob/main/Case_Study_04_Kelompok_F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h2><strong>Study Case 04 - Data Mining dan Business Intelligence: Korelasi dan Regresi</strong></h2></center>


# **Kelompok F**

Anggota :
 
1. Andini Assyahidah (2006571040)
2. Farah Tri Nurul Hayati (2006572882)
3. Ni Made Lidya Anggreni (2006522700)
4. Nurul Qomariah Abdillah (2006463811)
5. Peksyaji (2006522322)
6. Raistra Aurel (2006572876)
7. Yovela Natalia Yvone (2006463894)


# **Problem Statement**

ppp

# **Import Module**

In [1]:
import warnings; warnings.simplefilter('ignore')
import pandas as pd, seaborn as sns, matplotlib.pyplot as plt, numpy as np
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
import statsmodels.api as sm, scipy.stats as stats
from sklearn.datasets import load_boston
from sklearn.preprocessing import MinMaxScaler
plt.style.use('bmh'); sns.set()
"Done"

'Done'

# **Data**

In [ ]:
#Import data
df = pd.read_csv

In [ ]:
#Melihat informasi data
df.info()

Note:
- blabla
- blabla

# **Pre Processing**

In [ ]:
#Mengubah tipe data
df['p']=df['p'].astype('object')
df['x']=df['x'].astype('datetime64') 
df['y']=df['y'].astype('category')

Note:
- Kolom `p` diubah dari ini menjadi ini karena

In [ ]:
#Drop duplicates
df.shape #(lihat shape)
df.drop_duplicates()
df.shape #recheck shape

In [ ]:
df.info()

Note :

- Tipe data sudah sesuai dengan yang seharusnya.

# **EDA**

In [ ]:
#Korelasi dan Scatter Plot untuk melihat datanya
print('Covariance = ', np.cov(df.usia, df.tekanan_darah, ddof=0)[0][1])
print('Correlations = ', np.corrcoef(df.usia, df.tekanan_darah))
plt.scatter(df.usia, df.tekanan_darah)
plt.show()

# **Pemodelan Regresi**